In [3]:
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langgraph_supervisor import  create_supervisor
from langgraph.prebuilt import  create_react_agent
from openai import AzureOpenAI
import os
import azure.identity
from azure.identity import ClientSecretCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

In [1]:
try:
    keyVaultName = os.environ["KEY_VAULT_NAME"]
except KeyError:
    # Get input from user if not set
    keyVaultName = input("Please enter your Key Vault name: ")
    # Save for future cells in this session
    os.environ["KEY_VAULT_NAME"] = keyVaultName

In [4]:
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

In [5]:
azure_openai_endpoint=client.get_secret(name="aoai-endpoint").value
azure_openai_api_key=client.get_secret(name="aoai-api-key").value
azure_openai_api_version = "2024-02-15-preview"

In [6]:
model = AzureChatOpenAI(model="gpt-4o", api_key=azure_openai_api_key, api_version=azure_openai_api_version, azure_endpoint=azure_openai_endpoint)

In [15]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType
)

search_credential =AzureKeyCredential(client.get_secret(name="aisearch-key").value)
search_endpoint =client.get_secret(name="aisearch-endpoint").value
index_name = "csv-glossary-index"


def search_retrieval(user_input:str) -> str:
        """
        Search and retrieve answers from Azure AI Search.
        Returns:
            str
        """
        query = user_input
        search_result = ""
        search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=search_credential)
        vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=2, fields="vector", exhaustive=True)

        r = search_client.search(  
        search_text=query,
        vector_queries=[vector_query],
        select=["id", "notes", "chunk"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='my-semantic-config',
        query_caption=QueryCaptionType.EXTRACTIVE,
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=1
    )
        for result in r:  
            print(f"id: {result['id']}")  
            print(f"notes: {result['notes']}")  
            print(f"Score: {result['@search.score']}")  
            print(f"Content: {result['chunk']}")
            search_result += result['chunk']
        return search_result 

In [16]:
research_agent = create_react_agent(
    model=model,
    tools=[search_retrieval],
    name="search_expert",
    prompt="You are a world class researcher with access to an azure index. Use only the index response to respond."
)

In [17]:
result = research_agent.invoke(
    {
        "messages":[
            {
                "role": "user",
                "content": "what is agile?"
            }
        ]
    }
)

id: 99d593c454b5_aHR0cHM6Ly9ibG9ic3RvcmUwNS5ibG9iLmNvcmUud2luZG93cy5uZXQvZXhjZWwtZGF0YS9nbG9zc2FyeV9kYXRhc2V0LmNzdjs20_chunks_0
notes: Popular frameworks include Scrum and Kanban.
Score: 0.032786883413791656
Content: A project management and software development methodology focusing on iterative progress.


In [18]:
# Show agent's step by step response
for message in result["messages"]:
    print(message.pretty_print())

================================ Human Message =================================

what is agile?
None
================================== Ai Message ==================================
Name: search_expert
Tool Calls:
  search_retrieval (call_bIMQ56moR1gDzqeJXLYS82ci)
 Call ID: call_bIMQ56moR1gDzqeJXLYS82ci
  Args:
    user_input: agile
None
================================= Tool Message =================================
Name: search_retrieval

A project management and software development methodology focusing on iterative progress.
None
================================== Ai Message ==================================
Name: search_expert

Agile is a project management and software development methodology that emphasizes iterative progress. It is designed to be flexible and collaborative, allowing for adaptive planning and continuous improvement throughout the development process.
None
